In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Fri Aug 22 19:10:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!cd /content/drive/MyDrive/project2 && wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
# > /dev/null 2>&1
# rm -r food-101

--2025-08-22 19:11:33--  http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz [following]
--2025-08-22 19:11:33--  https://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4996278331 (4.7G) [application/x-gzip]
Saving to: ‘food-101.tar.gz’

food-101.tar.gz     100%[===================>]   4.65G  17.2MB/s    in 4m 22s  

2025-08-22 19:15:56 (18.2 MB/s) - ‘food-101.tar.gz’ saved [4996278331/4996278331]



In [ ]:
# !rm -r /content/drive/My\ Drive/project2/food-101/test

In [ ]:
os.path.isdir('/content/food-101/train')

True

In [ ]:
os.path.isdir('/content/food-101/test')

True

In [ ]:
###-------------------------------

In [ ]:
# --- Google Drive & dataset extraction ---
from google.colab import drive
drive.mount('/content/drive')

!tar -xvf /content/drive/MyDrive/project2/food-101.tar.gz > /dev/null 2>&1

# --- Prepare train/test folders from Food-101 splits ---
import os
from collections import defaultdict
import shutil
import stat

if not os.path.isdir('/content/food-101/test') or not os.path.isdir('/content/food-101/train'):

    def copytree(src, dst, symlinks=False, ignore=None):
        if not os.path.exists(dst):
            os.makedirs(dst)
            shutil.copystat(src, dst)
        lst = os.listdir(src)
        if ignore:
            excl = ignore(src, lst)
            lst = [x for x in lst if x not in excl]
        for item in lst:
            s = os.path.join(src, item)
            d = os.path.join(dst, item)
            if symlinks and os.path.islink(s):
                if os.path.lexists(d):
                    os.remove(d)
                os.symlink(os.readlink(s), d)
                try:
                    st = os.lstat(s)
                    mode = stat.S_IMODE(st.st_mode)
                    os.lchmod(d, mode)
                except:
                    pass  # lchmod not available
            elif os.path.isdir(s):
                copytree(s, d, symlinks, ignore)
            else:
                shutil.copy2(s, d)

    def generate_dir_file_map(path):
        dir_files = defaultdict(list)
        with open(path, 'r') as txt:
            files = [l.strip() for l in txt.readlines()]
            for f in files:
                dir_name, id = f.split('/')
                dir_files[dir_name].append(id + '.jpg')
        return dir_files

    train_dir_files = generate_dir_file_map('/content/food-101/meta/train.txt')
    test_dir_files  = generate_dir_file_map('/content/food-101/meta/test.txt')

    def ignore_train(d, filenames):
        subdir = d.split('/')[-1]
        return train_dir_files.get(subdir, [])

    def ignore_test(d, filenames):
        subdir = d.split('/')[-1]
        return test_dir_files.get(subdir, [])

    copytree('/content/food-101/images', '/content/food-101/test',  ignore=ignore_train)
    copytree('/content/food-101/images', '/content/food-101/train', ignore=ignore_test)
else:
    print('Train/Test files already copied into separate folders.')

print("Train dir exists:", os.path.isdir('/content/food-101/train'))
print("Test dir exists:", os.path.isdir('/content/food-101/test'))

# --- Imports (TF/Keras-compatible) ---
from __future__ import print_function, division


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train dir exists: True
Test dir exists: True


In [ ]:
import os
import numpy as np

from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3  # preprocess_input/decoded_predictions not used here
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, Flatten, AveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

# --- Data pipeline helpers ---
def setup_generator(train_path, test_path, batch_size, dimensions):
    # Using simple 1/255 rescale as in your original code.
    train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1.0/255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    test_datagen = ImageDataGenerator(rescale=1.0/255)

    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=dimensions,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    validation_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=dimensions,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, validation_generator

def load_image(img_path, dimensions, rescale=1.0/255):
    img = load_img(img_path, target_size=dimensions)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x *= rescale
    return x

# --- Build generators ---
shape = (224, 224, 3)  # keep as in your script
X_train, X_test = setup_generator('/content/food-101/train', '/content/food-101/test', 32, shape[:2])
num_classes = X_train.num_classes

# --- Model (InceptionV3 backbone) ---
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=shape))
x = base_model.output
x = AveragePooling2D(pool_size=(2, 2))(x)   # ✅ fixed
x = Dropout(0.5)(x)
x = Flatten()(x)
predictions = Dense(
    num_classes,
    activation='softmax',
    kernel_initializer='glorot_uniform',
    kernel_regularizer=l2(0.0005)
)(x)



Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
import os
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# --- Model ---
model = Model(inputs=base_model.input, outputs=predictions)

# --- Optimizer / compile ---
opt = SGD(learning_rate=0.1, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# --- Directories ---
log_dir = "/content/drive/MyDrive/project2/lan2_v"
os.makedirs(log_dir, exist_ok=True)  # make sure folder exists

# --- Callbacks ---
checkpointer = ModelCheckpoint(
    filepath=os.path.join(log_dir, "model4_lan2.{epoch:02d}-{val_loss:.2f}.keras"),
    verbose=1,
    save_best_only=True
)
csv_logger = CSVLogger(os.path.join(log_dir, "model4.log"))

def schedule(epoch):
    if epoch < 5:
        return 0.001
    elif epoch < 10:
        return 0.0002
    elif epoch < 15:
        return 0.00002
    else:
        return 0.0000005

lr_scheduler = LearningRateScheduler(schedule)

model.summary()

# --- (Optional) resume from a saved model if it exists ---
resume_path = "/content/drive/MyDrive/project2/lan2_v/model4.18-0.81.keras"  # updated to .keras
if os.path.exists(resume_path):
    model = load_model(resume_path)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    print(f"Resumed training from: {resume_path}")
else:
    print("No resume checkpoint found; training from scratch.")

# --- Train ---
model.fit(
    X_train,
    validation_data=X_test,
    epochs=50,
    steps_per_epoch=max(1, X_train.samples // X_train.batch_size),
    validation_steps=max(1, X_test.samples // X_test.batch_size),
    callbacks=[lr_scheduler, csv_logger, checkpointer]
)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_376 (Conv2D) │ (None, 111, 111,  │        864 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_376[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_376      │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_377 (Conv2D) │ (None, 109, 109,  │      9,216 │ activation_376[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_377[0][0]  │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_377      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_378 (Conv2D) │ (None, 109, 109,  │     18,432 │ activation_377[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_378[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_378      │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 54, 54,    │          0 │ activation_378[0… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_379 (Conv2D) │ (None, 54, 54,    │      5,120 │ max_pooling2d_16… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_379[0][0]  │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_379      │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_380 (Conv2D) │ (None, 52, 52,    │    138,240 │ activation_379[0… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_380[0][0]  │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_380      │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 22,630,277 (86.33 MB)

 Trainable params: 22,595,845 (86.20 MB)

 Non-trainable params: 34,432 (134.50 KB)

No resume checkpoint found; training from scratch.
Epoch 1/50
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 0s 547ms/step - accuracy: 0.2190 - loss: 3.6121
Epoch 1: val_loss improved from inf to 1.65064, saving model to /content/drive/MyDrive/project2/lan2_v/model4_lan2.01-1.65.keras
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 1457s 588ms/step - accuracy: 0.2191 - loss: 3.6117 - val_accuracy: 0.5939 - val_loss: 1.6506 - learning_rate: 0.0010
Epoch 2/50
   1/2367 ━━━━━━━━━━━━━━━━━━━━ 12:10 309ms/step - accuracy: 0.4688 - loss: 2.0999

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_loss did not improve from 1.65064
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 71s 30ms/step - accuracy: 0.4688 - loss: 2.0999 - val_accuracy: 0.5924 - val_loss: 1.6559 - learning_rate: 0.0010
Epoch 3/50
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 0s 505ms/step - accuracy: 0.5602 - loss: 1.8174
Epoch 3: val_loss improved from 1.65064 to 1.22739, saving model to /content/drive/MyDrive/project2/lan2_v/model4_lan2.03-1.23.keras
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 1346s 561ms/step - accuracy: 0.5602 - loss: 1.8173 - val_accuracy: 0.6941 - val_loss: 1.2274 - learning_rate: 0.0010
Epoch 4/50
   1/2367 ━━━━━━━━━━━━━━━━━━━━ 9:10 233ms/step - accuracy: 0.6562 - loss: 1.3424
Epoch 4: val_loss improved from 1.22739 to 1.22598, saving model to /content/drive/MyDrive/project2/lan2_v/model4_lan2.04-1.23.keras
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 68s 29ms/step - accuracy: 0.6562 - loss: 1.3424 - val_accuracy: 0.6948 - val_loss: 1.2260 - learning_rate: 0.0010
Epoch 5/50
2367/2367 ━━━━━━━━━━━━━━━━━━━━ 0s 506ms/step - accuracy

In [ ]:
# Evaluate on test dataset
loss, acc = model.evaluate(X_test, steps=X_test.samples // X_test.batch_size, verbose=1)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {acc:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 64/789 ━━━━━━━━━━━━━━━━━━━━ 1:00:48 5s/step - accuracy: 0.7120 - loss: 1.1318

KeyboardInterrupt: 

In [ ]:
# Predict on whole test set
y_pred_probs = model.predict(X_test, steps=X_test.samples // X_test.batch_size, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)

# True labels
y_true = X_test.classes[:len(y_pred)]

# Example: print first 10 predictions
for i in range(10):
    print(f"Image {i}: True={class_labels[y_true[i]]}, Pred={class_labels[y_pred[i]]}")


In [ ]:
from keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load trained model
model = load_model("/content/drive/MyDrive/project2/lan2_v/model4_lan2.15-0.85.keras")

# Path to test image
img_path = "/content/drive/MyDrive/project2/bg.jpg"

# Preprocess (must match training input shape, e.g. 224x224)
img = image.load_img(img_path, target_size=(224, 224))   # <-- fixed to 224
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) / 255.0   # normalize same way as training

# Predict
preds = model.predict(x)
predicted_class = np.argmax(preds, axis=1)[0]

# Map back to label names
class_labels = list(X_train.class_indices.keys())  # from training generator
print("Predicted class:", class_labels[predicted_class])


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted class: hamburger


In [ ]:
from keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load trained model
model = load_model("/content/drive/MyDrive/project2/lan2_v/model4_lan2.15-0.85.keras")

# Path to test image
# img_path = "/content/drive/MyDrive/project2/bg.jpg"
img_path = "/content/md.jpg"


# Preprocess (must match training input shape, e.g. 224x224)
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) / 255.0   # normalize same way as training

# Predict
preds = model.predict(x)[0]  # shape: (num_classes,)

# Get top-2 indices
top2_indices = preds.argsort()[-2:][::-1]

# Map back to label names (you need class_labels from training)
# Example: if you used ImageDataGenerator.flow_from_directory
# then class_labels = list(train_generator.class_indices.keys())
class_labels = list(X_train.class_indices.keys())

# Print top-2 classes with probabilities
for i in top2_indices:
    print(f"{class_labels[i]}: {preds[i]*100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
hummus: 27.10%
falafel: 15.77%


In [ ]:
import json

class_labels = list(X_train.class_indices.keys())
with open("/content/drive/MyDrive/project2/lan2_v/class_labels.json", "w") as f:
    json.dump(class_labels, f)
